In [9]:
#-*-coding:utf-8-*-

# import libraries
import cv2 as cv
import os
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as img

from PIL import Image

In [10]:
# base directory
base_dir = "/home/dmsai2/Desktop/DCC2022/data"
processed_dir = "/home/dmsai2/Desktop/DCC2022/pdata"

In [11]:
# load class list from directory
cls_list = os.listdir(base_dir)
cls_list

['L2_21',
 'L2_24',
 'L2_15',
 'L2_27',
 'L2_30',
 'L2_50',
 'L2_40',
 'L2_39',
 'L2_20',
 'L2_45',
 'L2_3',
 'L2_12',
 'L2_41',
 'L2_10',
 'L2_34',
 'L2_52',
 'L2_33',
 'L2_46',
 'L2_44',
 'L2_25']

In [12]:
# change class name to integer
clsdict = {}
for i, cls in enumerate(cls_list):
    clsdict[i] = cls
clsdict

{0: 'L2_21',
 1: 'L2_24',
 2: 'L2_15',
 3: 'L2_27',
 4: 'L2_30',
 5: 'L2_50',
 6: 'L2_40',
 7: 'L2_39',
 8: 'L2_20',
 9: 'L2_45',
 10: 'L2_3',
 11: 'L2_12',
 12: 'L2_41',
 13: 'L2_10',
 14: 'L2_34',
 15: 'L2_52',
 16: 'L2_33',
 17: 'L2_46',
 18: 'L2_44',
 19: 'L2_25'}

In [13]:
numdata = 0
numclass = 0
data_list = {}

for cls in cls_list:
    imlist = os.listdir(base_dir + "/" + cls)
    print(cls, end=' : ')
    print(len(imlist))
    numdata = numdata + len(imlist)
    data_list[cls] = imlist
    print("-"*12)

numclass = len(cls_list)

print("num of class :", numclass)
print("num of data :", numdata)

L2_21 : 410
------------
L2_24 : 782
------------
L2_15 : 1509
------------
L2_27 : 426
------------
L2_30 : 364
------------
L2_50 : 781
------------
L2_40 : 180
------------
L2_39 : 454
------------
L2_20 : 410
------------
L2_45 : 631
------------
L2_3 : 595
------------
L2_12 : 518
------------
L2_41 : 593
------------
L2_10 : 1893
------------
L2_34 : 419
------------
L2_52 : 382
------------
L2_33 : 6206
------------
L2_46 : 2214
------------
L2_44 : 547
------------
L2_25 : 6189
------------
num of class : 20
num of data : 25503


In [14]:
def cmyk2rgb(img):
    return img.convert("RGB")

In [15]:
def preprocess(img_dir):
    
    img = Image.open(img_dir)
    
    if img.mode == "RGB":
        pass
    else:
        img = img.convert("RGB")
    
    img = np.array(img)
    
    # if the image is png type,
    # remove the "alpha channel"
    # and change type from "float" to "np.uint8"
    # if img_dir[-3:] == "png":
    #     img = (plt.imread(img_dir)[:,:,0:3]*255).astype(np.uint8)
    # else: # jpg
    #     img = plt.imread(img_dir)[:,:,0:3]
    #     shp = img.shape
    #     
    #     if len(shp) == 2: # single channel
    #         img = np.reshape(img, (shp[0], shp[1], 1))
    #         img = np.concatenate((img, img, img), axis=2)
    #         # print(img.shape)
        
    # get channel
    x, y, c = img.shape
    
    # print(img.shape)
    
    # top to bottom
    cnt = 0
    for i in range(x):
        if np.all(img[i,:,0]==255) and np.all(img[i,:,1]==255) and np.all(img[i,:,2]==255):
            continue
        else:
            cnt = i
            break
    
    # print(cnt)
    for _ in range(cnt):
        img = np.delete(img, 0, axis=0)
    #print(img.shape)
    
    x, y, c = img.shape
    
    # bottom to top
    cnt = 0
    for i in range(x):
        if np.all(img[x-i-1,:,0]==255) and np.all(img[x-i-1,:,1]==255) and np.all(img[x-i-1,:,2]==255):
            continue
        else:
            cnt = i
            break
    
    #print(cnt)
    for _ in range(cnt):
        img = np.delete(img, -1, axis=0)
    #print(img.shape)
    
    x, y, c = img.shape
    
    # left to right
    cnt = 0
    for j in range(y):
        if np.all(img[:,j,0]==255) and np.all(img[:,j,1]==255) and np.all(img[:,j,2]==255):
            continue
        else:
            cnt = j
            break
    
    #print(cnt)
    for _ in range(cnt):
        img = np.delete(img, 0, axis=1)
    #print(img.shape)
    
    x, y, c = img.shape
    
    # right to left
    cnt = 0
    for j in range(y):
        if np.all(img[:,y-j-1,0]==255) and np.all(img[:,y-j-1,1]==255) and np.all(img[:,y-j-1,2]==255):
            continue
        else:
            cnt = j
            break
    
    #print(cnt)
    for _ in range(cnt):
        img = np.delete(img, -1, axis=1)
    #print(img.shape)
    
    x, y, c = img.shape
    
    # plt.imshow(img)
    
    # make to squre shape
    if x == y:
        timg = img
    elif x > y:
        #print("X")
        padding1 = np.ones(shape=(x, (x - y) // 2, 3), dtype=np.int8)*255
        padding2 = np.ones(shape=(x, x - y - (x - y) // 2, 3), dtype=np.int8)*255
        #print(padding1.shape, padding2.shape)
        timg = np.concatenate((padding1, img, padding2), axis=1)
        #print(timg.shape)
    else: # x < y
        #print("Y")
        padding1 = np.ones(shape=((y - x) // 2, y, 3), dtype=np.int8)*255
        padding2 = np.ones(shape=(y - x - (y - x) // 2, y, 3), dtype=np.int8)*255
        
        #print(padding1.shape, padding2.shape)
        
        timg = np.concatenate((padding1, img, padding2), axis=0)
        
        #print(timg.shape)
    
    return timg

In [16]:
# make directory in pdata folder
for label in data_list.keys():
    if os.path.isdir(processed_dir + "/" + label):
        print(processed_dir + "/" + label, "already exists")
    else:
        os.mkdir(processed_dir + "/" + label)
        print(processed_dir + "/" + label, "made")

/home/dmsai2/Desktop/DCC2022/pdata/L2_21 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_24 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_15 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_27 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_30 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_50 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_40 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_39 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_20 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_45 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_3 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_12 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_41 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_10 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_34 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_52 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_33 already exists
/home/dmsai2/Desktop/DCC2022/pdata/L2_46 already 

In [17]:
cnt = 0

error_list = []

# crop images and save data
for label in data_list.keys():
    print("LABEL :", label)
    for img_dir in data_list[label]:
        cnt = cnt + 1
        print(label, img_dir)
        
        if os.path.isfile(processed_dir + "/" + label + "/" + img_dir):
            print(f"file already exists! - {cnt}/{numdata} - {int((cnt / numdata * 100000))/1000}%")
        else:
            try:
                timg = preprocess(base_dir + "/" + label + "/" + img_dir)
                rimg = cv.resize(timg, dsize=(200, 200))
                rimg = Image.fromarray(np.uint8(rimg))
                img_name = img_dir[:-3] + "jpg"
                rimg.save(processed_dir + "/" + label + "/" + img_name)
                # cv.imwrite(processed_dir + "/" + label + "/" + img_dir, rimg)
                print(f"completed! - {cnt}/{numdata} - {int((cnt / numdata * 100000))/1000}%")
            except Exception as e:
                print(e)
                print("Error occured! at " + label + " : " + img_dir)
                error_list.append(label + " " + img_dir)
                
print("ERROR LIST")
print(error_list)

LABEL : L2_21
L2_21 mqxywrztgekfibkscbin.jpg
completed! - 1/25503 - 0.003%
L2_21 vqxqvmigfxzwhhphtgtw.jpg
completed! - 2/25503 - 0.007%
L2_21 lkgerbcjymjsuuxymhzh.jpg


KeyboardInterrupt: 

In [59]:
# # grayscale images
# cnt = 0

# d = {1:0, 2:0, 3:0, 4:0}

# # crop images and save data
# for label in data_list.keys():
#     print("LABEL :", label)
#     for img_dir in data_list[label]:
#         cnt = cnt + 1
#         testimg = plt.imread(base_dir + "/" + label + "/" + img_dir)
#         sh = testimg.shape
            
#         if (len(sh) != 3) or (sh[2] != 3):
#             print(label, img_dir, sh)
#             cnt += 1
    
# print(cnt)

# cnt = 0

# dpng = {1:0, 2:0, 3:0, 4:0, 5:0, 6:0}
# djpg = {1:0, 2:0, 3:0, 4:0, 5:0, 6:0}

# # crop images and save data
# for label in data_list.keys():
#     print("LABEL :", label)
#     for img_dir in data_list[label]:
#         cnt = cnt + 1
#         testimg = plt.imread(base_dir + "/" + label + "/" + img_dir)
#         sh = testimg.shape
        
#         if img_dir[-3:] == "png":
#             if len(sh) == 2:
#                 dpng[1] = dpng[1] + 1
#             else:
#                 dpng[sh[2]] = dpng[sh[2]] + 1
#         else:
#             if len(sh) == 2:
#                 djpg[1] = djpg[1] + 1
#             else:
#                 djpg[sh[2]] = djpg[sh[2]] + 1
            
# print(dpng)
# print(djpg)

LABEL : L2_21
L2_21 qnocxjtkdrqphkcuwbmv.png (700, 700, 4)
L2_21 vjzpaynrbiwivgqbvepg.png (700, 700, 4)
L2_21 cwxncrngxzvdqvgwhqkn.png (300, 300, 4)
L2_21 uexsptxhlmwazvvkofgm.png (1200, 1200, 4)
L2_21 sbzdhmtyvpxsqbgdfyjh.png (700, 700, 4)
L2_21 erdefxvaqbjiojjyikan.jpg (300, 300, 4)
L2_21 azuzshdtzvenfnemxqne.png (1200, 1200, 4)
L2_21 tnscxhgqespjcvjsajyx.png (700, 700, 4)
L2_21 tqnjuieetzpzncysmdnr.png (700, 700, 4)
L2_21 aplnyrgzunmonguhmnwj.png (300, 300, 4)
L2_21 vwboifkvaxjnepjcznwl.png (700, 700, 4)
L2_21 xbqeqaziclgtwighxlsq.png (1200, 1200, 4)
L2_21 dbhumtppgdivcdmkizoz.png (700, 700, 4)
L2_21 bfzbqjwvtfrqkwcidnfh.png (1200, 1200, 4)
L2_21 kentghplqrekihyokhri.png (700, 700, 4)
L2_21 lkuziffrkzolvvducwtg.png (700, 700, 4)
L2_21 dyfdrzhzchbzhpmbgfzl.png (700, 700, 4)
L2_21 ftwcjqrcqzjjhrhojxos.png (1200, 1200, 4)
L2_21 nqbzdmhhakgvvkepcvys.png (700, 700, 4)
L2_21 iaceuaonsylscimmsuug.png (300, 300, 4)
L2_21 izxdrhrweclrybaqwyrj.png (700, 700, 4)
L2_21 fuqrcynfdvflxbwqajut.png 

L2_27 hncldxcowynrzfwiresk.png (1200, 1200, 4)
L2_27 ctsndokyszxgxesdzwpl.png (300, 300, 4)
L2_27 mebcaoqtnigyeyopewdc.png (700, 700, 4)
L2_27 ombzisulajemvqgiwoqn.png (1200, 1200, 4)
L2_27 vgfxzycxgagoxhniouqo.png (700, 700, 4)
L2_27 sqsenoorgclmngyhfuri.png (300, 300, 4)
L2_27 jcfqnxnvjofdicebjift.png (300, 300, 4)
L2_27 eyzrzfusogqomffmbfyg.png (700, 700, 4)
L2_27 ntgfomjvbchwnaxfxdiq.png (700, 700, 4)
L2_27 euvaklwmzrfreliabhqq.png (1200, 1200, 4)
L2_27 somgkjnrmovrgvtesjcw.png (700, 700, 4)
L2_27 hqgxbwicgztgkfickmlt.png (700, 700, 4)
L2_27 fujgonxqztwmcivyxjfr.png (700, 700, 4)
L2_27 vhuutrixansfywbxszvv.jpg (700, 700, 4)
L2_27 xxtnhdkccpcefhvalqba.png (700, 700, 4)
L2_27 uwrtrwoaudsldadvlqxs.png (1200, 1200, 4)
L2_27 xityrmtakzqdwzvsoopl.png (300, 300, 4)
L2_27 snwjkevodabfgijceagk.png (300, 300, 4)
L2_27 zvgyvempfltkubyakcvk.png (300, 300, 4)
L2_27 ggcjydcfjctorgposttw.png (300, 300, 4)
L2_27 eeyawhkdheosoopnxxhx.png (300, 300, 4)
L2_27 cofygaqmqjcsnrcakubk.png (300, 300, 4)
L2

L2_40 ehynozxxksaljcfzqbjr.png (700, 700, 4)
L2_40 wiacnzrwjerucnrcwpog.png (700, 700, 4)
L2_40 fsngiivucvlfuftxcxzu.png (1200, 1200, 4)
L2_40 buafvcntgfpdjyjkunjr.png (700, 700, 4)
L2_40 tbcdkiguexpltbelpgpi.png (300, 300, 4)
L2_40 vbmxjzyaopqzsnmcuxye.png (700, 700, 4)
L2_40 iqajvkccmzeoukjbzawt.png (700, 700, 4)
L2_40 tqlgbpxvhmkggdkhdsma.png (700, 700, 4)
L2_40 lgimvveodjeduchrzlhx.png (300, 300, 4)
L2_40 dgptcjdydptfjloclopr.png (700, 700, 4)
L2_40 fzqtkcxeyvzaklztlmgz.png (700, 700, 4)
L2_40 zchkqakfgmrplnvohlbc.png (1200, 1200, 4)
L2_40 oiwpcglaxabqaxtexupj.png (700, 700, 4)
L2_40 orojejetvodigpwdmken.png (700, 700, 4)
L2_40 ldnsgpqallxltnarjuvj.png (700, 700, 4)
L2_40 xpgyryyybahxpzzuqfdd.png (300, 300, 4)
L2_40 addqarzmhtqkykardkec.png (300, 300, 4)
L2_40 mvjhvsqnhvzsmxeyptah.png (300, 300, 4)
L2_40 oisulzdczpwvpjlbblkg.png (700, 700, 4)
L2_40 ttmhlkhdpzozmcsmstgj.png (300, 300, 4)
L2_40 taygwsyqgybjgtchccoa.png (700, 700, 4)
L2_40 uxwmgsgrfxkjkvtotspv.png (700, 700, 4)
L2_40 

L2_45 faxlyvuyrxttxljeentu.png (300, 300, 4)
L2_45 ycjfjwstwiamlxzdgvhu.png (300, 300, 4)
L2_45 afowwqfnryurwlzjblua.png (700, 700, 4)
L2_45 jbvjumdlsvjssdkvrten.png (300, 300, 4)
L2_45 ihnsigwxssglbcxxteos.png (700, 700, 4)
L2_45 uaevideccjnctwbnjjvz.png (700, 700, 4)
L2_45 ducfwvnjwjgourqvepga.png (1200, 1200, 4)
L2_45 ryklomehzyewmozbixcl.png (300, 300, 4)
L2_45 bejjcnezarobjbllneaq.png (700, 700, 4)
L2_45 nktjpcvdenwbuutpkuos.png (700, 700, 4)
L2_45 fyuqvqgtfphaaxmpqgtr.jpg (300, 300, 4)
L2_45 spxfywyrpswwlezvvnoh.png (700, 700, 4)
L2_45 vbxwgzxuekpnzgdyldmz.png (700, 700, 4)
L2_45 opxowasvqtysxvgtqyvm.png (300, 300, 4)
L2_45 bdqhuoiayjtbakugsomd.png (300, 300, 4)
L2_45 cwwvgtepvjedmtcuohrf.png (700, 700, 4)
L2_45 fqtqwvvkdpgzzdyloupr.png (1200, 1200, 4)
L2_45 ualeivgtzpkttlgleymv.png (700, 700, 4)
L2_45 spsxjmwlugnerrwedjsy.png (700, 700, 4)
L2_45 zgzrffqojobfvaqtfenj.png (300, 300, 4)
L2_45 bcbyghzopdvlffponyzv.png (700, 700, 4)
L2_45 hhelunfabqhbeejkmpqr.png (700, 700, 4)
L2_45 

L2_10 pudwcsjmislvnhnnjsrv.png (300, 300, 4)
L2_10 snjublqxqvdqjtmywqec.png (700, 700, 4)
L2_10 zagdtzxkiofygvapxlnk.png (700, 700, 4)
L2_10 gcqxllzuudundqgxgyzj.png (700, 700, 4)
L2_10 ckiazuwmswqtllwzjcjk.png (700, 700, 4)
L2_10 gppklfbqzeudtjcmpxnt.png (700, 700, 4)
L2_10 tpomwfalzwqndxeglzjf.png (700, 700, 4)
L2_10 dfknwikbcpxpmbckjzgj.png (1200, 1200, 4)
L2_10 cncwxripwsqemmuzbudj.png (700, 700, 4)
L2_10 wpisiphohyzksjrgfnts.png (700, 700, 4)
L2_10 yimlhxljkigtetyplckf.png (300, 300, 4)
L2_10 zfyjkaeriuvuyfsnucqn.png (700, 700, 4)
L2_10 wbkxjlrsebobwmbxhoor.png (300, 300, 4)
L2_10 gtzxvcyysjhejhywmriy.png (700, 700, 4)
L2_10 mnuqjjeepvidoshwbdvw.png (700, 700, 4)
L2_10 raodbemcwulaylpkcqhi.png (300, 300, 4)
L2_10 pkuxtavwoyanprdbzpri.png (700, 700, 4)
L2_10 nakvelrxzmqmcafjlqlp.png (700, 700, 4)
L2_10 hqdqsatqrrdrwcjrqddb.png (300, 300, 4)
L2_10 womojylxaawtwsvqowhx.png (700, 700, 4)
L2_10 phdajmovegsivvclowgh.png (700, 700, 4)
L2_10 dlpijfylikdsrenncqtw.png (1200, 1200, 4)
L2_10 

L2_33 agfjpohncmryplkrftby.png (700, 700, 4)
L2_33 qbroiocpzxlkhgluisuw.png (700, 700, 4)
L2_33 dsgtjqzawohjpgmlzriy.png (1200, 1200, 4)
L2_33 jwuvfkvflnjuhzwrkgal.png (300, 300, 4)
L2_33 cuylykzdwcuhksumyeyc.png (300, 300, 4)


KeyboardInterrupt: 

In [61]:
# imgmodes = {}

# # crop images and save data
# for label in data_list.keys():
#     print("LABEL :", label)
#     for img_dir in data_list[label]:
#         timg = Image.open(base_dir + "/" + label + "/" + img_dir)
#         imgmode = timg.mode
        
#         if imgmode in imgmodes.keys():
#             imgmodes[imgmode] = imgmodes[imgmode] + 1
#         else:
#             imgmodes[imgmode] = 1
            
# print(imgmodes)

LABEL : L2_21
LABEL : L2_24
LABEL : L2_15
LABEL : L2_27
LABEL : L2_30
LABEL : L2_50
LABEL : L2_40
LABEL : L2_39
LABEL : L2_20
LABEL : L2_45
LABEL : L2_3
LABEL : L2_12
LABEL : L2_41
LABEL : L2_10
LABEL : L2_34
LABEL : L2_52
LABEL : L2_33
LABEL : L2_46
LABEL : L2_44
LABEL : L2_25
{'RGB': 24318, 'RGBA': 1127, 'CMYK': 42, 'P': 10, 'L': 6}
